In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:

# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.in',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}


In [ ]:

url="https://www.amazon.in/gp/bestsellers"
response=requests.get(url,headers=headers)
response.status_code



In [ ]:

soup=BeautifulSoup(requests.get(url,headers=headers).text,'lxml')
link=[]
count=0
for i in soup.find_all("div",{'class':"_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-small__nleKL"}):
    link.append("https://www.amazon.in"+i.a.get("href"))
    count=count+1


In [ ]:
    
prod_name=[]
prod_link=[]
i=0
for i in range(len(link)):
    soup=BeautifulSoup(requests.get(link[i],headers=headers).text,'lxml')
    
    for i in soup.find_all("div",{'class':"p13n-sc-uncoverable-faceout"}):
        prod_name.append(i.span.div.text)
        prod_link.append("https://www.amazon.in"+i.a.get("href")) 


In [ ]:
    
pro_link = []
count=0
for li in prod_link:
    try:
        soup = BeautifulSoup(requests.get(li, headers=headers).text, 'html.parser')
        print(f"prod_link: {li}")
        lin = soup.find('a', attrs={'data-hook': 'see-all-reviews-link-foot'})
        count +=1
        if lin and lin.get("href"):
            print('https://www.amazon.in' +lin.get("href"))
            pro_link.append('https://www.amazon.in' + lin.get("href"))
        else:
            print("No reviews link found.")
    except Exception as e:
        print(f"Error processing link {li}: {e}")


In [ ]:

print(len(pro_link))


In [ ]:

reviews = []
count=0
j=0
for j in range(len(pro_link)):
    for k in range(10):
        response = pro_link[j] + '&pageNumber=' + str(k)
        
        print( count)
        count +=1
        print(response)
        soup=BeautifulSoup(requests.get(response,headers=headers).text,'lxml')
        for i in soup.findAll("div", {'data-hook': "review"}):
            try:
                name = i.select_one('[class="a-profile-name"]').text.strip()
            except Exception as e:
                name = 'N/A'

            try:
                stars = i.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
            except Exception as e:
                stars = 'N/A'   

            try:
                title = i.select_one('[data-hook="review-title"]').text.strip().split('\n')[1]
            except Exception as e:
                title = 'N/A'

            try:
                # Convert date str to dd/mm/yyy format
                date = i.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]

            except Exception as e:
                date = 'N/A'

            try:
                description = i.select_one('[data-hook="review-body"]').text.strip()
            except Exception as e:
                description = 'N/A'
                
            try:
                verification = i.select_one('[data-hook="avp-badge"]').text.strip()
            except Exception as e:
                verification = 'N/A'
                
            # create Dictionary with all review data 
            data_dict = {
                'product':prod_name[j],
                'Name': name,
                'Stars': stars,
                'Title': title,
                'Date': date,
                'Verification': verification,
                'Description': description
            }

            # Add Dictionary in master empty List
            reviews.append(data_dict)


In [ ]:

reviews


In [ ]:

len(reviews)


In [ ]:

review_data=pd.DataFrame.from_dict(reviews)
pd.set_option('max_colwidth',800)

In [ ]:
review_data.to_csv('scrapingReview.csv')